In [1]:
import pandas as pd
import plotly.express as px
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc

In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

importing dataset

In [45]:
data = pd.read_excel("permsnewdata.xlsx", header=0)
data.head()

,num_cars,num_pedestrians,patience,num_lanes,light_interval,road_density,avg_waiting_cars,avg_crowd_size,avg_speed_cars,no._stopped_cars
0,5,5,0.2,1,0.5,0.706684,136.487841,64.333939,17.744244,96.453820
1,5,5,0.2,1,1.0,0.380718,121.103942,10.185717,1.693121,46.106393
2,5,5,0.2,1,1.5,0.041003,132.340874,26.587662,42.803609,5.426313
3,5,5,0.2,1,2.0,0.048675,59.273655,61.475978,46.999891,2.885116
4,5,5,0.2,2,0.5,0.528469,95.814285,80.096772,88.154918,50.634851


## Graphs

### Graphs for tab 1

### Graphs for tab 2-1

In [4]:
def create_plot_crowd(time="afternoon", density="medium"):
    df = data[data["time_of_day"] == time]
    df = df[df["population_density"] == density]

    crowd_heat_df = df[["number_of_lanes", "ratio", "avg_crowd_size"]]
    crowd_lanes = df[["number_of_lanes", "avg_crowd_size"]].groupby("number_of_lanes").mean()
    crowd_ratio = df[["ratio", "avg_crowd_size"]].groupby("ratio").mean()

    fig = make_subplots(
        rows=2, cols=2,
        column_widths=[0.5, 0.5],
        row_heights=[0.5, 0.5],
        specs=[[{"type": "bar"}, {"type": "heatmap", "rowspan": 2}],
           [{"type": "bar"}, None]])
    
    fig.add_trace(
    go.Heatmap(x = crowd_heat_df["number_of_lanes"],
        y =  crowd_heat_df["ratio"],
        z =  crowd_heat_df["avg_crowd_size"]
        ),
    row=1, col=2,
    )

    fig.add_trace(
    go.Bar(x = crowd_lanes.index,
        y = crowd_lanes["avg_crowd_size"]
        ),
    row=1, col=1,
    )

    fig.add_trace(
    go.Bar(x = crowd_ratio.index,
        y = crowd_ratio["avg_crowd_size"]
        ),
    row=2, col=1,
    )

    fig.update_xaxes(title_text="Number of Lanes", row=1, col=2)
    fig.update_xaxes(title_text="Number of Lanes", row=1, col=1)
    fig.update_xaxes(title_text="Green-Red Ratio", row=2, col=1)

    fig.update_yaxes(title_text="Green-Red Ratio", row=1, col=2)
    fig.update_yaxes(title_text="Average Crowd Size", row=1, col=1)
    fig.update_yaxes(title_text="Average Crowd Size", row=2, col=1)
    fig.update_layout(
        showlegend=False
    )


    return fig


In [5]:
create_plot_crowd()

### Graphs for tab 2-2

In [35]:
car_marker_df = data[["num_lanes", "light_interval", "avg_waiting_cars", "no._stopped_cars"]]
car_heat_df = data[["num_lanes", "light_interval", "avg_speed_cars"]].pivot(index='num_lanes', columns='light_interval',values="avg_speed_cars")
car_ratio = data[["light_interval", "avg_waiting_cars"]].groupby("light_interval").mean()
car_lanes = data[["num_lanes", "avg_waiting_cars"]].groupby("num_lanes").mean()
    

ValueError: Index contains duplicate entries, cannot reshape

In [29]:
px.scatter(
    x=car_marker_df["light_interval"],
    y=car_marker_df["num_lanes"],
    color=car_marker_df["avg_waiting_cars"],
    size=car_marker_df["no._stopped_cars"]
)

In [33]:
px.imshow(car_heat_df["avg_speed_cars"], 
          x = car_heat_df["num_lanes"],
          y =  car_heat_df["light_interval"])

ValueError: px.imshow only accepts 2D single-channel, RGB or RGBA images. An image of shape (8000,) was provided. Alternatively, 3- or 4-D single or multichannel datasets can be visualized using the `facet_col` or/and `animation_frame` arguments.

In [36]:
px.line(x = car_lanes.index,
        y = car_lanes["avg_waiting_cars"]
        )

In [38]:
px.bar(x = car_ratio.index,
        y = car_ratio["avg_waiting_cars"]
)

In [56]:
def create_plot_car():

    car_marker_df = data[["num_lanes", "light_interval", "avg_waiting_cars", "no._stopped_cars"]]
    car_heat_df = data[["num_lanes", "light_interval", "avg_speed_cars"]]
    car_ratio = data[["light_interval", "avg_waiting_cars"]].groupby("light_interval").mean()
    car_lanes = data[["num_lanes", "avg_waiting_cars"]].groupby("num_lanes").mean()
    

    fig = make_subplots(
        rows=4, cols=1,
        vertical_spacing=0.25)

    fig.add_trace(
    go.Scatter(
        mode='markers',
        x=car_marker_df["light_interval"],
        y=car_marker_df["num_lanes"],
        marker=dict(
            color=car_marker_df["avg_waiting_cars"],
            size=car_marker_df["no._stopped_cars"]
        )
    ),
    row=1, col=1,
    )

    fig.add_trace(
    go.Heatmap(x = car_heat_df["num_lanes"],
        y =  car_heat_df["light_interval"],
        z =  car_heat_df["avg_speed_cars"]
        ),
    row=2, col=1,
    )

    fig.add_trace(
    go.Line(x = car_lanes.index,
        y = car_lanes["avg_waiting_cars"]
        ),
    row=3, col=1,
    )

    fig.add_trace(
    go.Bar(x = car_ratio.index,
        y = car_ratio["avg_waiting_cars"]
        ),
    row=4, col=1,
    )

    fig.update_xaxes(title_text="Light Interval", row=4, col=1)
    fig.update_xaxes(title_text="Number of Lanes", row=3, col=1)
    fig.update_xaxes(title_text="Number of Lanes", row=2, col=1)
    fig.update_xaxes(title_text="Light Interval", row=1, col=1)
    
    fig.update_yaxes(title_text="Average Waiting Time", row=4, col=1)
    fig.update_yaxes(title_text="Average Waiting Time", row=3, col=1)
    fig.update_yaxes(title_text="Light Interval", row=2, col=1)
    fig.update_yaxes(title_text="Number of Lanes", row=1, col=1)
    fig.update_layout(
        showlegend=False
    )

    return fig

In [57]:
create_plot_car()

/opt/homebrew/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




### Graphs for tab 2-3

## The APP

In [6]:
app=JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP, 'https://codepen.io/chriddyp/pen/bWLwgP.css'])

### Tab 1

In [7]:
content1 = dbc.Row([
    dbc.Col([
        html.H3("Placeholder for inputs"),
        html.H3("Placeholder for outputs")
    ], width = 5),
    dbc.Col([
        html.H3("Placeholder for Graph"),
    ])
]
)

### Tab 2

In [8]:
content2 = dcc.Tabs(id="graph-tabs", children=[
            dcc.Tab([dcc.Graph(id = "graph_crowd", figure=create_plot_crowd())],label='Pedestrians'),
            dcc.Tab([dcc.Graph(id = "graph_crowd2", figure=create_plot_crowd())],label='Cars'),
            dcc.Tab(label='Compare'),
        ],  vertical=True, parent_style={'float': 'left'})

### Overall

In [9]:
tabs = dbc.Tabs([
    dbc.Tab([
        content1
    ], label = "Tab 1"), 
    dbc.Tab([
        content2
    ], label = "Tab 2")
])

In [10]:
sidebar = html.Div([
    html.H6("Number of Cars"),
    dcc.Slider(id = "number-of-cars", min = 0, max = 50, step = 5, value=10, 
        tooltip={"placement": "bottom", "always_visible": True}),
    html.H6("Number of Pedestrians"),
    dcc.Slider(id = "number-of-pedestrians", min = 0, max = 50, step = 5, value=10, 
        tooltip={"placement": "bottom", "always_visible": True}),
    html.H6("Max patience"),
    dcc.Slider(id = "max-patience", min = 0, max = 1, step = 0.2, value=1, 
        tooltip={"placement": "bottom", "always_visible": True}),
])

In [11]:



app.layout = html.Div(
    children=[
        html.H1("Title"),
        dbc.Row([ 
            dbc.Col([
                sidebar
            ], width = 2),
            dbc.Col([
                tabs
            ], width = 10)
        ])
    ],
    style={"margin-left": "20px", "margin-right": "20px", "margin-top": "15px"}
)


In [12]:
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
